In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [14]:
file_path = "coin_Bitcoin.csv" 
df = pd.read_csv(file_path)

In [15]:
df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
df = df.sort_values("Date")

In [16]:
df = df.dropna()

features = ["High", "Low", "Open", "Volume", "Marketcap"]
target = "Close"



In [17]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[features + [target]])



In [18]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) 
        y.append(data[i+seq_length, -1])  
    return np.array(X), np.array(y)

seq_length = 10 
X, y = create_sequences(df_scaled, seq_length)



In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [20]:
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(seq_length, X.shape[2])),
    LSTM(100, return_sequences=True),
    LSTM(50),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  
])

/Users/tejaskalyankar/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [22]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 5.0316e-04 - mae: 0.0136 - val_loss: 0.0066 - val_mae: 0.0454
Epoch 2/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.4592e-04 - mae: 0.0070 - val_loss: 0.0054 - val_mae: 0.0492
Epoch 3/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.3221e-04 - mae: 0.0068 - val_loss: 0.0142 - val_mae: 0.0736
Epoch 4/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.0656e-04 - mae: 0.0064 - val_loss: 0.0099 - val_mae: 0.0572
Epoch 5/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.7773e-05 - mae: 0.0060 - val_loss: 0.0206 - val_mae: 0.0924
Epoch 6/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.1127e-05 - mae: 0.0060 - val_loss: 0.0059 - val_mae: 0.0427
Epoch 7/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.7510e-05 - mae: 0.0054 - val_loss: 0.0044 - val_mae: 0.0366
Epoch 8/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.9012e-05 - mae: 0.0055 - val_loss: 0.0055 - val_mae: 0.0413
Epoch 9/50
149/149 ━━━━

In [23]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Model Loss: {loss:.4f}, Model MAE: {mae:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012 - mae: 0.0122     
Model Loss: 0.0049, Model MAE: 0.0377


In [24]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-Squared (R2 Score): {r2:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Mean Squared Error (MSE): 0.0049
Mean Absolute Error (MAE): 0.0377
R-Squared (R2 Score): 0.9354
